## Advanced Data Analysis using Amazon AgentCore Bedrock Code Interpreter- Tutorial(Strands)
This tutorial demonstrates how to create an AI agent that performs advanced data analysis through code execution using Python. We use Amazon Bedrock AgentCore Code Interpreter to run code that is generated by the LLM.

This tutorial demonstrates how to use AgentCore Bedrock Code Interpreter to:
1. Set up a sandbox environment
2. Configure a strands based agent that performs advanced data analysis by generating code based on the user query
3. Execute code in a sandbox environment using Code Interpreter
4. Display the results back to the user

## Prerequisites
- AWS account with Bedrock AgentCore Code Interpreter access
- You have the necessary IAM permissions to create and manage code interpreter resources
- Required Python packages installed(including boto3, bedrock-agentcore & strands)
- IAM role should have permissions to invoke models on Amazon Bedrock
 - Access to Claude Haiku 4.5 & Claude Sonnet 3.5 models in the US Oregon (us-west-2) region

## Your IAM execution role should have the following IAM policy attached

~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## How it works

The code execution sandbox enables agents to safely process user queries by creating an isolated environment with a code interpreter, shell, and file system. After a Large Language Model helps with tool selection, code is executed within this session, before being returned to the user or Agent for synthesis.

![architecture local](code-interpreter.png)

## 1. Setting Up the Environment

First, let's import the necessary libraries and initialize our Code Interpreter client.

The default session timeout is 900 seconds(15 minutes). However, we start the session with a slightly session timeout duration of 1200 seconds(20 minutes), since we will perform detailed analysis on our data

In [1]:
!pip install --upgrade -r requirements.txt

In [7]:
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
from strands import Agent, tool
from strands.models import BedrockModel
import json
import pandas as pd
from typing import Dict, Any, List

# Initialize the Code Interpreter within a supported AWS region.
code_client = CodeInterpreter('us-west-2')
code_client.start(session_timeout_seconds=1200)

'01KCCNKTHR9VJWS3H7WZ2H1JH7'

## 2. Reading Local Data File

Now we'll read the contents of our sample data file. The file consists of random data with 4 columns: Name, Preferred_City, Preferred_Animal, Preferred_Thing and ~ 300,000 records.

We will analyze this file using an agent little later, to understand distributions and outliers

In [8]:

df_data = pd.read_csv("samples/data.csv")
df_data.head()

,Name,Preferred_City,Preferred_Animal,Preferred_Thing
0,Betty Ramirez,Dallas,Elephant,Sofa
1,Jennifer Green,Naples,Bee,Shirt
2,John Lopez,Helsinki,Zebra,Wallet
3,Susan Gonzalez,Beijing,Chicken,Phone
4,Jennifer Wright,Buenos Aires,Goat,Wallet


In [9]:
def read_file(file_path: str) -> str:
    """Helper function to read file content with error handling"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

data_file_content = read_file("samples/data.csv")

## 3. Preparing Files for Sandbox Environment

We'll create a structure that defines the files we want to create in the sandbox environment.

In [10]:
files_to_create = [
                {
                    "path": "data.csv",
                    "text": data_file_content
                }]

## 4. Creating Helper Function for Tool Invocation

This helper function will make it easier to call sandbox tools and handle their responses. Within an active session, you can execute code in supported languages (Python, JavaScript), access libraries based on your dependencies configuration, generate visualizations, and maintain state between executions.

In [11]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools

    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool

    Returns:
        Dict[str, Any]: JSON formatted result
    """
    response = code_client.invoke(tool_name, arguments)
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. Write data file to Code Sandbox

Now we'll write our data file into the sandbox environment and verify they were created successfully.

In [12]:
# Write files to sandbox
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# Verify files were created
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

Writing files result:
{"content": [{"type": "text", "text": "Successfully wrote all 1 files"}], "isError": false}

Files in sandbox:
{"content": [{"type": "resource_link", "uri": "file:///log", "name": "log", "description": "Directory"}, {"type": "resource_link", "uri": "file:///run", "name": "run", "description": "Directory"}, {"type": "resource_link", "uri": "file:///.ipython", "name": ".ipython", "description": "Directory"}, {"type": "resource_link", "mimeType": "text/csv", "uri": "file:///data.csv", "name": "data.csv", "description": "File"}], "isError": false}


## 6. Perform Advanced Analysis using Strands based Agent

Now we will configure an agent to perform data analysis on the data file that we uploaded into the sandbox(above)

### 6.1 System Prompt Definition
Define the behavior and capabilities of the AI assistant. We instruct our assistant to always validate answers through code execution and data based reasoning.

In [13]:
SYSTEM_PROMPT = """You are a helpful AI assistant that validates all answers through code execution using the tools provided. DO NOT Answer questions without using the tools

VALIDATION PRINCIPLES:
1. When making claims about code, algorithms, or calculations - write code to verify them
2. Use execute_python to test mathematical calculations, algorithms, and logic
3. Create test scripts to validate your understanding before giving answers
4. Always show your work with actual code execution
5. If uncertain, explicitly state limitations and validate what you can

APPROACH:
- If asked about a programming concept, implement it in code to demonstrate
- If asked for calculations, compute them programmatically AND show the code
- If implementing algorithms, include test cases to prove correctness
- Document your validation process for transparency
- The sandbox maintains state between executions, so you can refer to previous results

TOOL AVAILABLE:
- execute_python: Run Python code and see output

RESPONSE FORMAT: The execute_python tool returns a JSON response with:
- sessionId: The sandbox session ID
- id: Request ID
- isError: Boolean indicating if there was an error
- content: Array of content objects with type and text/data
- structuredContent: For code execution, includes stdout, stderr, exitCode, executionTime

For successful code execution, the output will be in content[0].text and also in structuredContent.stdout.
Check isError field to see if there was an error.

Be thorough, accurate, and always validate your answers when possible."""

### 6.2 Code Execution Tool Definition
Next we define the function as tool that will be used by the Agent as tool, to run code in the code sandbox. We use the @tool decorator to annotate the function as a custom tool for the Agent.

Within an active code interpreter session, you can execute code in supported languages (Python, JavaScript), access libraries based on your dependencies configuration, generate visualizations, and maintain state between executions.

In [14]:
#Define and configure the code interpreter tool
@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""

    if description:
        code = f"# {description}\n{code}"

    #Print generated Code to be executed
    print(f"\n Generated Code: {code}")


    # Call the Invoke method and execute the generated code, within the initialized code interpreter session
    response = code_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False
    })
    for event in response["stream"]:
        return json.dumps(event["result"])

### 6.3 Agent Configuration
We create and configure an agent using the Strands SDK. We provide it the system prompt and the tool we defined above to execute generate code.

We use the Claude Haiku 4.5 model and we specify the [Cross-region Inference (CRIS)](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html) profile id

In [15]:
model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
model= BedrockModel(model_id=model_id)

#configure the strands agent including the model and tool(s)
agent=Agent(
    model=model,
        tools=[execute_python],
        system_prompt=SYSTEM_PROMPT,
        callback_handler=None)

## 7. Agent Invocation and Response Processing
We invoke the agent with our query and process the agent's response


Note: Async execution requires running in an async environment

## 7.1 Query to perform Exploratory Data Analysis(EDA)

Let's start with a query which instructs the agent to perform exploratory data analysis on the data file in the code sandbox environment

In [16]:
query = "Load the file 'data.csv' and perform exploratory data analysis(EDA) on it. Tell me about distributions and outlier values."

# Invoke the agent asynchcronously and stream the response
response_text = ""
async for event in agent.stream_async(query):
    if "data" in event:
        # Stream text response
        chunk = event["data"]
        response_text += chunk
        print(chunk, end="")

I'll load the file 'data.csv' and perform a comprehensive exploratory data analysis for you.
 Generated Code: 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load the CSV file
df = pd.read_csv('data.csv')

# Display basic information
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"Shape: {df.shape}")
print(f"\nColumn Names and Types:")
print(df.dtypes)
print(f"\nFirst few rows:")
print(df.head())
print(f"\nDataset Info:")
df.info()

Let me try without seaborn:
 Generated Code: 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Load the CSV file
df = pd.read_csv('data.csv')

# Display basic information
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"Shape: {df.shape}")
print(f"\nColumn Names and Types:")
print(df.dtypes)
print(f"\nFirst few rows:")
print(df.head(10))
print(f"\nLast few rows:")
print(df.tail())
print(f"\nMissing Valu

## 7.2 Query to extract information

Now, let's instruct the agent to extract specific information from the data file in the code sandbox environment

In [17]:
query = "Within the file 'data.csv', how many individuals with the first name 'Kimberly' have 'Crocodile' as their favourite animal?"

# Invoke the agent asynchcronously and stream the response
response_text = ""
async for event in agent.stream_async(query):
    if "data" in event:
        # Stream text response
        chunk = event["data"]
        response_text += chunk
        print(chunk, end="")

I'll search the data for individuals with the first name 'Kimberly' who have 'Crocodile' as their favorite animal.
 Generated Code: 
import pandas as pd

# Load the CSV file
df = pd.read_csv('data.csv')

# Extract first name from the Name column
df['First_Name'] = df['Name'].str.split().str[0]

# Filter for Kimberly with Crocodile as favorite animal
result = df[(df['First_Name'] == 'Kimberly') & (df['Preferred_Animal'] == 'Crocodile')]

count = len(result)

print("=" * 80)
print("SEARCH RESULTS")
print("=" * 80)
print(f"\nQuery: Individuals with first name 'Kimberly' AND favorite animal 'Crocodile'")
print(f"\nNumber of matches: {count}")

if count > 0:
    print(f"\nDetails of matching records:")
    print(result[['Name', 'First_Name', 'Preferred_City', 'Preferred_Animal', 'Preferred_Thing']])
else:
    print("\nNo records found matching the criteria.")

# Additional statistics
print("\n" + "=" * 80)
print("VERIFICATION & ADDITIONAL STATS")
print("=" * 80)
kimberly_count = len(df[df['

## 8. Cleanup

Finally, we'll clean up by stopping the Code Interpreter session. Once finished using a session, the session should be shopped to release resources and avoid unnecessary charges.

In [18]:
# Stop the Code Interpreter session
code_client.stop()
print("Code Interpreter session stopped successfully!")

Code Interpreter session stopped successfully!
